# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [1]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [2]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [3]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


c:\Users\setday\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400">
- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400">
- D: <img src=https://i.ibb.co/Y47Mr83b/mdp-d.png caption="D" width="400">
- E: <img src=https://i.ibb.co/Kjt1Xhmf/mdp-e.png caption="E" width="400">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [4]:
states = set(range(3))
actions = set(range(1))

print(f'{states=} | {actions=}')

T = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}
print(f'Transition function {T=}')

A_mdp = states, actions, T

states_B = set(range(4))
actions_B = set(range(3))
T_B = {
    (0, 0): 1, (0, 1): 2, (0, 2): 0,
    (1, 0): 1, (1, 1): 1, (1, 2): 1,
    (2, 0): 2, (2, 1): 2, (2, 2): 2,
    (3, 0): 3, (3, 1): 3, (3, 2): 3
}
B_mdp = states_B, actions_B, T_B

states_C = set(range(4))
actions_C = set(range(2))
T_C = {
    (0, 0): 1, (0, 1): 2,
    (1, 0): 1, (1, 1): 3,
    (2, 0): 3, (2, 1): 2,
    (3, 0): 3, (3, 1): 3
}
C_mdp = states_C, actions_C, T_C

states_D = set(range(3))
actions_D = set(range(1))
T_D = {
    (0, 0): [0.2, 0.8, 0.0],
    (1, 0): [0.0, 0.3, 0.7],
    (2, 0): [0.0, 0.0, 1.0],
}
D_mdp = states_D, actions_D, T_D

states_E = set(range(4))
actions_E = set(range(2))
T_E = {
    (0, 0): [0.0, 0.8, 0.2, 0.0], (0, 1): [0.0, 0.1, 0.9, 0.0],
    (1, 0): [0.0, 0.2, 0.0, 0.8], (1, 1): [0.0, 0.0, 1.0, 0.0], # or ..., (1, 1): 2
    (2, 0): [0.0, 1.0, 0.0, 0.0], (2, 1): [0.0, 0.0, 0.3, 0.7], # or (2, 0): 1, ...
    (3, 0): [0.0, 0.0, 0.0, 1.0], (3, 1): [0.0, 0.0, 0.0, 1.0], # or (3, 0): 3, (3, 1): 3
}
E_mdp = states_E, actions_E, T_E

states={0, 1, 2} | actions={0}
Transition function T={(0, 0): 1, (1, 0): 2, (2, 0): 2}


Попробуйте записать функцию перехода в матричном виде:

In [5]:
T_A_Matrix = np.array([
    [1],
    [2],
    [2]
])
T_B_Matrix = np.array([
    [1, 2, 0],
    [1, 1, 1],
    [2, 2, 2],
    [3, 3, 3]
])
T_C_Matrix = np.array([
    [1, 2],
    [1, 3],
    [3, 2],
    [3, 3]
])
T_D_Matrix = None # Since it has non-deterministic transitions
T_E_Matrix = None # Since it has non-deterministic transitions

T_A_Matrix_prob = np.array([
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 0.0, 1.0]
])
T_B_Matrix_prob = np.array([
    [0.0, 1/3, 1/3, 1/3],
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
]) # Use any action with uniform probability
T_C_Matrix_prob = np.array([
    [0.0, 1/2, 1/2, 0.0],
    [0.0, 1/2, 0.0, 1/2],
    [0.0, 0.0, 1/2, 1/2],
    [0.0, 0.0, 0.0, 1.0]
]) # Use any action with uniform probability
T_D_Matrix_prob = np.array([
    [0.2, 0.8, 0.0],
    [0.0, 0.3, 0.7],
    [0.0, 0.0, 1.0],
])
T_E_Matrix_prob = np.array([
    [0.0, 0.8 * 1/2 + 0.1 * 1/2, 0.2 * 1/2 + 0.9 * 1/2, 0.0],
    [0.0, 0.2 * 1/2 + 0.0 * 1/2, 0.0 * 1/2 + 1.0 * 1/2, 0.8 * 1/2 + 0.0 * 1/2],
    [0.0, 1.0 * 1/2 + 0.0 * 1/2, 0.0 * 1/2 + 0.3 * 1/2, 0.0 * 1/2 + 0.7 * 1/2],
    [0.0, 0.0, 0.0, 1.0]
])

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [6]:
def get_after_N_steps(T: np.ndarray, s: np.ndarray, N: int) -> np.ndarray:
    """
    T - transition function in matrix form (SxA)
    s - start state
    N - number of steps
    """
    for _ in range(N):
        s = s @ T
    return s

A_inf = get_after_N_steps(T_A_Matrix_prob, np.array([1, 0, 0]), 1000)
B_inf = get_after_N_steps(T_B_Matrix_prob, np.array([1, 0, 0, 0]), 1000)
C_inf = get_after_N_steps(T_C_Matrix_prob, np.array([1, 0, 0, 0]), 1000)
D_inf = get_after_N_steps(T_D_Matrix_prob, np.array([1, 0, 0]), 1000)
E_inf = get_after_N_steps(T_E_Matrix_prob, np.array([1, 0, 0, 0]), 1000)

print(f'{A_inf=}\n{B_inf=}\n{C_inf=}\n{D_inf=}\n{E_inf=}')

A_inf=array([0., 0., 1.])
B_inf=array([0.        , 0.33333333, 0.33333333, 0.33333333])
C_inf=array([0.00000000e+000, 9.33263619e-302, 9.33263619e-302, 1.00000000e+000])
D_inf=array([0., 0., 1.])
E_inf=array([0.00000000e+000, 1.60806811e-204, 1.69048035e-204, 1.00000000e+000])


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [7]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))
    
# Optimal agent for MDP A
class OptimalAgentA:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        return 0  # There is only one action
    
# Optimal agent for MDP B
class OptimalAgentB:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        return 1  # There is only one non-terminal state without self-loop, so we are going to best reward
    
# Optimal agent for MDP C
class OptimalAgentC:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        if state in [0, 1]: return 1  # Going without loops
        if state == 2: return 0  # Going to terminal state
        return 0  # There is no actions in terminal state

## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [8]:
R = {
    (0, 0): -0.1,
    (1, 0): 1.0,
    (2, 0): 0.0
}
print(R)

A_mdp = *A_mdp, R
print(A_mdp)

R_B = {
    (0, 0): 0.0, (0, 1): 1.0, (0, 2): 0.0,
    (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0,
    (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0,
    (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0,
}
B_mdp = *B_mdp, R_B

R_C = {
    (0, 0): -0.1, (0, 1): +0.1,
    (1, 0): -0.5, (1, 1): +1.0,
    (2, 0): -0.1, (2, 1): -0.5,
    (3, 0): 0.0, (3, 1): 0.0,
}
C_mdp = *C_mdp, R_C

D_mdp = *D_mdp, R
E_mdp = *E_mdp, R_C

{(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}
({0, 1, 2}, {0}, {(0, 0): 1, (1, 0): 2, (2, 0): 2}, {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0})


## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [9]:
def run_episode(mdp):
    states, actions, T, R = mdp
    agent = Agent(actions)

    s = 0
    tau = []
    for _ in range(5):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

    return tau

run_episode(A_mdp)

[(0, 0, -0.1), (1, 0, 1.0), (2, 0, 0.0), (2, 0, 0.0), (2, 0, 0.0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

In [12]:
class Env:
    def __init__(self, states, actions, T, R, terminal_states, initial_states=None):
        self.states = states
        self.actions = actions
        self.T = T
        self.R = R
        self.terminal_states = terminal_states
        self.initial_states = initial_states

        self.current_state = None

    def reset(self):
        if self.initial_states is not None:
            self.current_state = np.random.choice(self.initial_states)
        else:
            self.current_state = np.random.choice(self.states)
        return self.current_state

    def step(self, action):
        if self.current_state is None:
            raise ValueError("Environment not reset. Call reset() before step().")

        # Get the next state based on the transition probabilities
        trans = self.T[self.current_state, action]
        if isinstance(trans, int):
            next_state = trans
        else:
            next_state = np.random.choice(list(self.states), p=trans)
        reward = self.R[self.current_state, action]
        termination = next_state in self.terminal_states

        self.current_state = next_state
        return next_state, reward, termination

In [25]:
def run_episode(mdp):
    states, actions, T, R = mdp
    agent = Agent(actions)
    env = Env(states, actions, T, R, terminal_states={2}, initial_states=[0])

    s = env.reset()
    tau = []
    termination = False
    for _ in range(5):
        a = agent.act(s)
        s_next, r, termination = env.step(a)

        tau.append((s, a, r))
        s = s_next

        if termination:
            break

    return tau

run_A = run_episode(A_mdp)
run_B = run_episode(B_mdp)
run_C = run_episode(C_mdp)
run_D = run_episode(D_mdp)
run_E = run_episode(E_mdp)

print(run_A)
print(run_B)
print(run_C)
print(run_D)
print(run_E)

[(0, 0, -0.1), (1, 0, 1.0)]
[(0, 0, 0.0), (1, 2, 0.0), (1, 0, 0.0), (1, 1, 0.0), (1, 2, 0.0)]
[(0, 1, 0.1)]
[(0, 0, -0.1), (0, 0, -0.1), (0, 0, -0.1), (1, 0, 1.0), (1, 0, 1.0)]
[(0, 0, -0.1), (1, 1, 1.0)]


### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [31]:
def compute_G_by_run(tau, gamma=0.9):
    G = 0.0
    for t, (_, _, r) in enumerate(tau):
        G += (gamma ** t) * r
    return G

def compute_return(mdp, n_runs=1000, gamma=0.9):
    return np.mean([compute_G_by_run(run_episode(mdp), gamma) for _ in range(n_runs)])

G_hat_A = compute_return(A_mdp)
G_hat_B = compute_return(B_mdp)
G_hat_C = compute_return(C_mdp)
G_hat_D = compute_return(D_mdp)
G_hat_E = compute_return(E_mdp)

print(f'{G_hat_A=}\n{G_hat_B=}\n{G_hat_C=}\n{G_hat_D=}\n{G_hat_E=}')

G_hat_A=0.8000000000000002
G_hat_B=0.46738620000000003
G_hat_C=0.19424729999999998
G_hat_D=1.0397854700000002
G_hat_E=0.10194600000000001
